In [ ]:
# Juypter Labs Notebooked used to train, evaluate a Deeplabcut Pose estimation model. As well as run inference with model, optimise/filter predictions
# and finally annotate the predictions on the original video/images
# This notebook was used on Goole Colab, but can be run locally with the correct dependencies installed and with devices has cuda/mps support

In [ ]:
# Import section
import deeplabcut
print("Import Complete")

In [ ]:
# Mount Google drive section
# Place Zipped project here
# User may wish to provide zipped or unzipper project folder
from google.colab import drive
drive.mount('/content/drive')
print("Mount Complete")

In [ ]:
# Extract Zippped Project Section
# Can omit lines if unzipped project folder is provided
# Just define project_name and project_dir
import zipfile
import os
import shutil

# Define paths
uploaded_zip_path = "/content/drive/MyDrive/DLC_Data/SimpleDLC_TF-Steph-2025-04-24.zip"
extract_to = "/content/drive/MyDrive/DLC_Data"
project_name = "SimpleDLC_TF-Steph-2025-04-24"
project_dir = os.path.join(extract_to, project_name)

# Clear previous extraction (if any)
if os.path.exists(project_dir):
    shutil.rmtree(project_dir)
os.makedirs(extract_to, exist_ok=True)

# Unzip into clean directory
with zipfile.ZipFile(uploaded_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Project extracted successfully.")
print("Project path:", project_dir)


In [ ]:
# Load Paths sections
project_path = "/content/drive/MyDrive/DLC_Data/SimpleDLC_TF-Steph-2025-04-24"
config_path = f"{project_path}/config.yaml"
print("Config Loaded")

Config Loaded


In [ ]:
# Dataset Creation Section
deeplabcut.create_training_dataset(config_path)

In [ ]:
# Extra Section to resize images
# Can possibly omit but images are trained on the cropped images of the full frame image
# Use section as template to account for this for future training if needed
import yaml

# Path to pose_cfg.yaml file
pose_cfg_path = "/content/drive/MyDrive/DLC_Data/SimpleDLC_TF-Steph-2025-04-24/dlc-models/iteration-0/FinalDLC_TFApr24-trainset95shuffle1/train/pose_cfg.yaml"

# Load current config
with open(pose_cfg_path, "r") as f:
    cfg = yaml.safe_load(f)

# Make safe edits
cfg['global_scale'] = 1.0
cfg['min_input_size'] = 32

# Save back the updated config
with open(pose_cfg_path, "w") as f:
    yaml.dump(cfg, f)

print("pose_cfg.yaml updated with:")
print(" - global_scale = 1.0")
print(" - min_input_size = 32")


In [ ]:
# Training Loop Section
try:
    deeplabcut.train_network(
        config=config_path,
        shuffle=1,
        displayiters=500,
        saveiters=1000,
        maxiters=20000,
        gputouse=0
    )
except Exception as e:
    print("\nTraining likely completed but caught thread-related exception:")
    print(e)

print("\nTraining Finished.")

# Added due to thread error attributed to Google Collab and Python threads double check files to find snapshots to verify completion otherwise omit
# if you plan to you different machine to train

In [ ]:
# Evaluation Section
try:
    # Evaluate the trained network
    deeplabcut.evaluate_network(
        config_path,
        plotting=True,
        show_errors=True
    )
    print("\nEvaluation Complete.")

except Exception as e:
    print("\nEvaluation finished, but caught a thread-related exception :")
    print(e)

model_folder = os.path.join(
    os.path.dirname(config_path),
    "dlc-models",
    "iteration-0"
)

if os.path.exists(model_folder):
    print(f"\nFound model folder: {model_folder}")
    print("Contents (snapshots/checkpoints):")
    for root, dirs, files in os.walk(model_folder):
        for file in files:
            if file.startswith("snapshot") and (file.endswith(".index") or file.endswith(".meta")):
                print(f" - {file}")
else:
    print("\nWARNING: Model folder not found. Please double-check your training.")


# Added due to thread error attributed to Google Collab and Python threads double check files to find snapshots to verify completion otherwise omit
# if you plan to you different machine to train

In [ ]:
# Load videos to run inference
video_folder = "/content/drive/MyDrive/DLC_Data/videos"
print("Videos Loaded")

In [ ]:
# Inference section

video_paths = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.endswith(".avi")]

# Run inference
deeplabcut.analyze_videos(
    config_path,
    video_paths,
    videotype='.avi',
    save_as_csv=True,
    # gputouse=0 # 0 = 1 GPU, 1 = 2 GPU, "Omit" or None = CPU
)
print("Inference Complete")

In [ ]:
# Remove weak correlation and optimise inference data beforehand
deeplabcut.extract_outlier_frames(
    config_path,
    video_paths,
    automatic=True,
    outlieralgorithm="jump",
    alpha=0.05,
    p_bound=0.01
)

In [ ]:
# Annotate Videos based on inferenced/predicated labels from trained model
deeplabcut.create_labeled_video(
    config_path,
    video_paths,
    draw_skeleton=True,
    save_frames=True,
    videotype='.avi'
)

print("Labelled Videos Complete")